In [0]:
%pip uninstall -y great_expectations          # quita cualquier versión previa
%pip install --upgrade --force-reinstall "great_expectations==0.18.11"
dbutils.library.restartPython()               # reinicia el intérprete

Found existing installation: great-expectations 1.4.5
Uninstalling great-expectations-1.4.5:
  Successfully uninstalled great-expectations-1.4.5
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for great_expectations==0.18.11 from https://files.pythonhosted.org/packages/84/25/d207236a7261d618b388def9e21debe936f2d117b7f618e03e0ea38b059c/great_expectations-0.18.11-py3-none-any.whl.metadata
  Obtaining dependency information for altair<5.0.0,>=4.2.1 from https://files.pythonhosted.org/packages/18/62/47452306e84d4d2e67f9c559380aeb230f5e6ca84fafb428dd36b96a99ba/altair-4.2.2-py3-none-any.whl.metadata
  Using cached altair-4.2.2-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for Click>=7.1.2 from https://files.pythonhosted.org/packages/85/32/10bb5764d90a8eee674e9dc6f4db6a0ab47c8c4d0d83c27f7c39ac415a4d/click-8.2.1-py3-none-any.whl.metadata
  Obtaining dependency in

In [0]:
import pandas as pd
import numpy as np

# 1. Leer CSV
df = pd.read_csv("/Workspace/Users/azzure_roma_29@hotmail.com/ENTREGAS/validation_data_to_be_shared.csv")

# 2. Eliminar columnas con más del 50% de valores perdidos
umbral = 0.5
df = df.loc[:, df.isnull().mean() < umbral]

# 3. Eliminar columnas con un solo valor único (sea número o texto)
cols_unico = [col for col in df.columns if df[col].nunique(dropna=True) <= 1]
df = df.drop(columns=cols_unico)

print("Shape después de limpieza de columnas:", df.shape)

# 4. Imputación eficiente de datos faltantes, con manejo de errores
def imputar_datos(df):
    try:
        for col in df.columns:
            if pd.api.types.is_numeric_dtype(df[col]):
                # Imputar con la mediana para evitar sesgos
                mediana = df[col].median()
                if np.isnan(mediana):  # Si toda la columna está vacía (raro, pero posible)
                    continue
                df[col] = df[col].fillna(mediana)
            elif pd.api.types.is_object_dtype(df[col]):
                # Imputar con el valor más frecuente (moda)
                moda = df[col].mode(dropna=True)
                if not moda.empty:
                    df[col] = df[col].fillna(moda[0])
                else:
                    # Si no hay moda, rellena con string vacío (seguro)
                    df[col] = df[col].fillna("")
            elif pd.api.types.is_datetime64_any_dtype(df[col]):
                # Imputar fechas con la más frecuente o dejar nulo si no hay
                moda = df[col].mode(dropna=True)
                if not moda.empty:
                    df[col] = df[col].fillna(moda[0])
    except Exception as e:
        print(f"⚠️ Error imputando columna '{col}':", str(e))
    return df

df = imputar_datos(df)

print("¿Todavía quedan nulos?", df.isnull().sum().sum())
print(df.head())

Shape después de limpieza de columnas: (41792, 1139)
¿Todavía quedan nulos? 0
   account_number  onus_attribute_1  ...  bureau_enquiry_49  bureau_enquiry_50
0          100001           34000.0  ...                0.0               32.0
1          100002          100000.0  ...                0.0                0.0
2          100003          130000.0  ...                0.0               12.0
3          100004          100000.0  ...                0.0                1.0
4          100005           53000.0  ...                0.0               18.0

[5 rows x 1139 columns]


In [0]:
import great_expectations as gx

# 1. Convertir el DataFrame de pandas a un objeto de Great Expectations
gx_df = gx.dataset.PandasDataset(df)

# 2. Generar expectativas automáticas para TODAS las columnas
for col in df.columns:
    
    # 2a. Siempre: no permitir nulos
    gx_df.expect_column_values_to_not_be_null(col)
    
    # 2b. Lógica según el tipo de dato
    if pd.api.types.is_numeric_dtype(df[col]):
        # Percentiles 1 % y 99 % como rango razonable
        low, high = df[col].quantile([0.01, 0.99])
        gx_df.expect_column_values_to_be_between(col, min_value=low, max_value=high)
        
    elif pd.api.types.is_object_dtype(df[col]):
        # Si hay pocas categorías, limitar al conjunto observado
        n_unique = df[col].nunique(dropna=True)
        if 0 < n_unique <= 20:
            allowed = df[col].dropna().unique().tolist()
            gx_df.expect_column_values_to_be_in_set(col, allowed)

    # (Opcional) podrías añadir reglas para fechas o texto largo aquí.

# 3. Ejecutar la validación
result = gx_df.validate()

# 4. Reporte sencillo en consola
print("\n────────  RESUMEN  ────────")
print("¿Validación exitosa? ", result.success)
print("Expectativas evaluadas:", len(result.results))

# 5. Mostrar detalles solo de las expectativas que fallaron
if not result.success:
    print("\n❌ Expectativas que FALLARON:")
    for r in result.results:
        if not r.success:
            etype = r.expectation_config["expectation_type"]
            col   = r.expectation_config["kwargs"].get("column")
            print(f" - {etype}  →  columna: {col}")
else:
    print("✅ Todas las expectativas se cumplieron.")



────────  RESUMEN  ────────
¿Validación exitosa?  False
Expectativas evaluadas: 2278

❌ Expectativas que FALLARON:
 - expect_column_values_to_be_between  →  columna: account_number
 - expect_column_values_to_be_between  →  columna: onus_attribute_1
 - expect_column_values_to_be_between  →  columna: transaction_attribute_1
 - expect_column_values_to_be_between  →  columna: transaction_attribute_2
 - expect_column_values_to_be_between  →  columna: transaction_attribute_3
 - expect_column_values_to_be_between  →  columna: transaction_attribute_4
 - expect_column_values_to_be_between  →  columna: transaction_attribute_5
 - expect_column_values_to_be_between  →  columna: transaction_attribute_6
 - expect_column_values_to_be_between  →  columna: transaction_attribute_7
 - expect_column_values_to_be_between  →  columna: transaction_attribute_8
 - expect_column_values_to_be_between  →  columna: transaction_attribute_9
 - expect_column_values_to_be_between  →  columna: transaction_attribute_10